In [1]:
# 앞에 몽고db랑 연결하는 코드 작성!!!
# 아래는 chatbot 에 purchaselist와 연결되어있는상태
# db연결하고 아래 코드 실행하면 된다


In [2]:
import pymongo

In [3]:
# MongoClient
# default:localhost로 몽고디비 서버에 원격으로 연결해주는 객체
conn = pymongo.MongoClient()
conn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
#주피터에서 chatbot_deep 과 연결
chatdeep = conn.chatbot_deep
chatdeep

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'chatbot_deep')

In [5]:
#chatbot_deep디비의 chatbot 콜렉션에 연결
chatbot = chatdeep.chatbot
chatbot

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'chatbot_deep'), 'chatbot')

In [6]:
# read documents 해보자.
#전체검색
docs = chatbot.find()
docs

In [9]:
#user_id이 admin인 사람 검색
docs = chatbot.find({"user_id":"admin"})
docs

In [10]:
for doc in docs:
    print(doc)

{'_id': ObjectId('62b1299753032bce2459ac3a'), 'info_idx': '4', 'product_idx': '6', 'payment_method': '신용카드', 'user_id': 'admin', 'payment_total': '49900', 'recipient_info': 'rtavnerlu', 'recipient_addr': '경기도 부천', 'purchase_product': '라이 맥시 롱 원피스 black[블랙]', 'purchase_quantity': '5', 'payment_date': '2022-04-19 23:48:00'}
{'_id': ObjectId('62b1299753032bce2459ac3b'), 'info_idx': '5', 'product_idx': '7', 'payment_method': '신용카드', 'user_id': 'admin', 'payment_total': '49900', 'recipient_info': 'ddisbreyfy', 'recipient_addr': '서울시 왕십리 ', 'purchase_product': '여성 트위스트 셔츠 원피스 - 에스프레소 / DR254LF588495', 'purchase_quantity': '3', 'payment_date': '2022-04-20 23:32:00'}
{'_id': ObjectId('62b1299753032bce2459ac3c'), 'info_idx': '6', 'product_idx': '8', 'payment_method': '신용카드', 'user_id': 'admin', 'payment_total': '39900', 'recipient_info': 'pwoodings5p', 'recipient_addr': '서울시 동작구 12-2', 'purchase_product': '여성 트위스트 원피스 - 레몬 글레이즈 / W221DR254LF588500', 'purchase_quantity': '5', 'payment_date': '20

In [12]:
def purchase_search():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbotchatdeep
    purchaselist = chatdeep.chatbot
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    id = input('아이디를 입력하세요 >> ')
    print('1(총 구매 목록), 2(구매 상품), 3(총 구매 가격), 4(결제 건수), 5(특정 시간에 따른 구매 목록)')
    while True:
        print()
        check = int(input('확인하시려는 항목을 선택하세요 >> '))
        print()
        if check == 1:
            search_list = list[list['user_id']==id]
            print('고객님의 구매목록은 다음과 같습니다')
            search_list = pd.DataFrame(search_list, 
                                             columns=['purchase_product', 'payment_method', 'payment_total', 'payment_date'])
            print()
            print(search_list.to_string(index=False))
        elif check == 2:
            search_product = list[list['user_id']==id]['purchase_product']
            print('고객님이 현재까지 구매하신 상품은 다음과 같습니다.')
            print()
            print(','.join(search_product.to_list()))
        elif check == 3:
            search_tprice = list[list['user_id']==id]['payment_total'].astype(int).sum()
            print('고객님의 총 구매 가격은 ' + str(search_tprice) + "입니다")
        elif check == 4:
            list['payment_total'] = list['payment_total'].astype(int)
            total_count = list[list['user_id']=='admin'][['payment_method', 'payment_total']]
            search_payment_method_credit = total_count[total_count['payment_method'] == '신용카드']['payment_method'].unique()[0]
            search_payment_total_credit = total_count[total_count['payment_method'] == '신용카드']['payment_total'].sum()
            search_payment_method_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_method'].unique()[0]
            search_payment_total_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_total'].sum()
            print(
                  '조회하신 id에 대한 결제 건수는, ' + search_payment_method_credit + ": " + str(search_payment_total_credit) +"원, " +
                  search_payment_method_noaccount + ": " + str(search_payment_total_noaccount) + "원입니다."
            )
        elif check == 5:
            search_date = input('조회하려는 날짜를 선택하세요 (YYYY-mm-dd 형식으로 입력해주세요) >> ')
            search_date_list = list[(list['user_id']==id) & (list['payment_date'].str.contains(search_date))]
            print('조회하신 날짜에 대한 구매목록은 다음과 같습니다.')
            print()
            print(search_date_list.to_string(index=False))
        else:
            print('챗봇을 종료합니다. 이용해주셔서 감사합니다')
            break

In [ ]:
purchase_search()